In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [8]:
#| hide
from langGraph_swarm.agent import *

In [9]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

# langGraph_swarm

> A framework that simoulates the Agency swarm library using langgraph

This file will become your README and also the index of your documentation.

## Install

```sh
pip install langGraph_swarm
```

## How to use

Fill me in please! Don't forget code examples:

# Tool for summarize pdf

In [12]:
from typing import Annotated, List, Tuple, Union, Dict
from pydantic import BaseModel, Field

from langchain.tools import BaseTool, StructuredTool, Tool
from langchain_core.tools import tool
from langchain.document_loaders import UnstructuredURLLoader, UnstructuredHTMLLoader, SeleniumURLLoader
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain.document_loaders import UnstructuredFileLoader
import random
import instructor
import openai

# This executes code locally, which can be unsafe
# python_repl_tool = PythonREPLTool()

In [13]:
client = instructor.patch(openai.OpenAI())

In [14]:
class Summarize(BaseModel):
    """Extract and summarize the text to keep the important information for marketing purposes."""
    # summary: str = Field(..., description="""A detailed and professional summary of the text.""")
    # value_prop: List[str] = Field(..., description="The list of value propositions of the technology presented.")
    # key_points: List[str] = Field(..., description="List of the key points and ideas presented in hte text.")
    answer: str = Field(..., description="""A short sentence explaining what was done and the result obtained.""")

In [15]:
@tool('summarize_pdf', return_direct=False)
def summarize_pdf(file: str, task: str):
    """Given the file path, and the task, accomplish the task using only the information from the file."""
    loader = UnstructuredFileLoader(file)
    doc = loader.load()
    response: Summarize = client.chat.completions.create(
        model = 'gpt-4-0125-preview',
        temperature=0.2,
        response_model=Summarize,
        messages=[
            {
                "role": "system",
                "content": """You are an expert summarizing and extracting information from any file.""",
            },
            {
                "role": "user",
                "content": f"""Use the content below:\n{doc}""",
            },

        ],
        )
    return response.model_dump()

# Tool to Use content from file to answer

In [16]:
class FileResponse(BaseModel):
    response: str = Field(..., description="The answer to the question/task made.")

In [17]:
@tool('answer_from_file', return_direct=False)
def answer_from_file(file: str, task: str):
    """Given the task, use the content of the file to answer and retrieve the information needed."""
    loader = UnstructuredFileLoader(file)
    doc = loader.load()
    response: FileResponse = client.chat.completions.create(
        model = 'gpt-4-0125-preview',
        temperature=0.2,
        response_model=FileResponse,
        messages=[
            {
                "role": "system",
                "content": """You are an expert summarizing and extracting information from tech notes.""",
            },
            {
                "role": "user",
                "content": f"""{task}""",
            },
            {
                "role": "user",
                "content": f"""Use the content below:\n{doc}""",
            },

        ],
        )
    return response.model_dump()

# Tool to search information in Pubmed

In [18]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

In [19]:
@tool('search_pubmed', return_direct=False)
def search_pubmed(query:str):
    """Tool to search scientific publications related to the query."""
    pubmed = PubmedQueryRun()
    return pubmed.run(query)

In [20]:
class FileResponse(BaseModel):
    answer: str = Field(..., description="The answer to the question/task made.")

In [21]:
@tool('answer_from_file', return_direct=False)
def answer_from_file(file: str, task: str):
    """Given the task, use the content of the file to answer and retrieve the information needed."""
    loader = UnstructuredFileLoader(file)
    doc = loader.load()
    response: FileResponse = client.chat.completions.create(
        model = 'gpt-4-0125-preview',
        temperature=0.2,
        response_model=FileResponse,
        messages=[
            {
                "role": "system",
                "content": """You are an expert summarizing and extracting information from tech notes.""",
            },
            {
                "role": "user",
                "content": f"""{task}""",
            },
            {
                "role": "user",
                "content": f"""Use the content below:\n{doc}""",
            },

        ],
        )
    return response.model_dump()

# Tool for review and translate the post

In [22]:
class Post(BaseModel):
    """Correction and translation of LinkedIn posts to make it engaging, fun and trustful"""
    corrected: str = Field(..., description="A corrected version of the post, decreasing the emojis, making the language understandable for everyone and adding some call to actions")
    answer: str = Field(..., description="""A short sentence explaining what was done and the status of the task.""")
    # portuguese: List[str] = Field(..., description="The corrected post, translated to portugues")
    # spanish: List[str] = Field(..., description="The corrected post, translated to spanish")

In [23]:
@tool('corrector', return_direct=False)
def corrector(post: str)->Dict:
    """This tool recieve a post for LinkedIn and do some modification and translations to make it ready to post."""

    response: Post = client.chat.completions.create(
        model = 'gpt-4-1106-preview',
        temperature=0.5,
        response_model=Post,
        messages=[
            {
                "role": "system",
                "content": """You are an experienced marketing editor. Your work is to read, correct and modify the post.
                Make the language understandable, make it more engaging, and add some call to action.""",
            },
            {
                "role": "user",
                "content": f"""{post}""",
            },

        ],
        )
    
    return response.model_dump()

# Tool for URL text

In [24]:
@tool('get_url_text', return_direct=False)
def get_url_text(url:str)->str:
    """Function that load a url, extract the html and return the text from the website"""
    loader = SeleniumURLLoader([url])
    docs = loader.load()
    return docs[0].page_content

# Tool for SEO generation 

In [25]:
from langchain_community.utilities.dataforseo_api_search import DataForSeoAPIWrapper

In [27]:
import os
os.environ["DATAFORSEO_LOGIN"] = "eb0cbsc6j@mozmail.com"
os.environ["DATAFORSEO_PASSWORD"] = "b92f6748ba83c900"

seo_w = DataForSeoAPIWrapper()

@tool('seo', return_direct=False)
def seo(query:str):
    "Search SEO optimized sentences"
    return seo_w.run(query)
    
# seo = Tool(
#     name="SEO search",
#     description="Search for sentences with SEO to a topic",
#     func=seo_w.run,
# )

# Tool for search google

In [28]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
@tool('search_google', return_direct=False)
def search_google(query:str)->str:
    """Tool that search in google the query"""
    return search.run(query)

ImportError: google-api-python-client is not installed. Please install it with `pip install google-api-python-client>=2.100.0`

# Tool for search google

In [29]:
duck = DuckDuckGoSearchRun()

ImportError: Could not import duckduckgo-search python package. Please install it with `pip install -U duckduckgo-search`.

In [6]:
pubmed = PubmedQueryRun()

In [30]:
agent = Agent(role="You are a Web Researcher",goal="use the your tools to answer the question", backstory="your life depends on it", tools=[pubmed, get_url_text, summarize_pdf], verbose=True)

In [32]:
agent.execute_task(task="find information about FOPPR and ists application in biophysics")



> Entering new AgentExecutor chain...

Invoking: `pub_med` with `FOPPR biophysics`


No good PubMed Result was found
Invoking: `get_url_text` with `{'url': 'https://www.google.com/search?q=FOPPR+biophysics'}`
responded: It appears that searching for "FOPPR biophysics" on PubMed did not yield useful results. Given the specificity of the topic and the initial lack of information, it seems FOPPR might be a niche or highly specialized topic within the field of biophysics. To proceed, I will attempt to find information about FOPPR and its application in biophysics through a general web search. Please bear with me as I conduct this search.



ImportError: unstructured package not found, please install it with `pip install unstructured`